In [1]:
# You may need to install these libraries. Run this cell first to see
# conda install -c conda-forge dash
# conda install -c plotly jupyter-dash
import os
from os import path

import pandas as pd

import numpy as np
from datetime import datetime 
import time

import requests

import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.graph_objects as go

In [2]:
#Read Data and formatting
MainDataSet = pd.read_csv("FinalData.csv")
MainDataSet.index =  pd.to_datetime(MainDataSet['Date'] , format = '%d/%m/%Y %H:%M')
df = MainDataSet.drop(['Unnamed: 0', 'Date'], axis=1)
years = pd.DatetimeIndex(df.index).year.unique()
years = [str(x) for x in years] # get the years only in an array
Dyears = { i : years[i] for i in range(0, len(years) ) } # to get in the format {0: '2014', 1: '2015', 2: '2016', 3: '2017', 4: '2018', 5: '2019', 6: '2020'}

the_year = 2016
run_df = df[(df.index > '1/01/{} 00:00'.format(str(years[0]))) & (df.index <= '11/01/{} 00:00'.format(str(the_year )))]
run_df

,temp,wdsp,wddir,wetb,dewpt,rhum,Sitting_Room,BoilerOn,BoilerOff,Temperature,Sp_Temperature
Date,,,,,,,,,,,
2014-01-11 00:00:00,3.5,3,120,2.5,0.8,82,20.746667,NaN,NaN,NaN,NaN
2014-01-11 01:00:00,3.2,8,210,2.8,2.1,92,20.137273,NaN,NaN,NaN,NaN
2014-01-11 02:00:00,2.6,10,250,2.3,1.8,94,19.825000,NaN,NaN,NaN,NaN
2014-01-11 03:00:00,2.3,6,230,2.1,1.6,95,19.553333,NaN,NaN,NaN,NaN
2014-01-11 04:00:00,1.5,7,220,1.3,0.8,95,19.340909,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2016-10-31 20:00:00,10.2,11,30,8.5,6.6,78,23.277500,0.0,99.0,23.483333,17.000000
2016-10-31 21:00:00,9.9,9,30,8.4,6.6,80,22.694545,0.0,100.0,23.616667,17.000000
2016-10-31 22:00:00,9.4,7,30,8.4,7.3,86,22.287500,0.0,100.0,23.271429,8.428571


In [3]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
############################### Declare  Figs #######################################################
wind = run_df['wddir'].value_counts().to_frame()
wind = wind.sort_index(axis=0, level=None, ascending=True,)
thedate = run_df.index
descript = {'wdsp':'Wind Speed', 'wddir':'Wind Direction in Degrees', 'wetb': 'WetBulb Temperature','dewpt':'Dew Point',\
              'rhum': 'Relative Humididty', 'Sitting_Room':'Inside Temperature','BoilerOn':'Boiler On',\
               'Sp_Temperature':'Stat Set Point', 'temp':'Outside Temperature'}

# First Figure with Year and Sitting room Temp
fig = go.Figure()
selected=['temp','Sitting_Room']
for i in selected:
    y0 = run_df['{}'.format(i)]  
    thename = i
    fig.add_trace(go.Scatter(x=thedate, y=y0,mode='lines',name= descript.get(i))),

# Figure2 Bipolar wind speed distribution
fig2 = go.Figure()
fig2.add_trace(go.Barpolar(r = wind.wddir,marker_color='rgb(33,208,57)'))
fig2.update_layout(title='Wind Speed Distribution',font_size=16,legend_font_size=16,polar_angularaxis_rotation=90,)

# Figure3: 3 dimensional plot with chosen variables
fig3 = go.Figure(data=[go.Mesh3d(x=(run_df['temp']),
                   y=(run_df['wdsp']),
                   z=(run_df['wddir']),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])
fig3.update_layout(scene = dict(
                    xaxis_title='Inside Temperature',
                    yaxis_title='Wind Speed',
                    zaxis_title='Wind Direction'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

data = requests.get("http://api.openweathermap.org/data/2.5/weather?lat=54.25847553809279&lon=-8.48109570196112&appid=27f70634bcb390ac526971b077f7acfd")
dataJ = data.json()
Wdis = dataJ["weather"][0]['description']
cTemp = round(dataJ['main']['temp'] - 273.15)
cWindsp = round(dataJ['wind']['speed'])
cWinddir = round(dataJ['wind']['deg'])


In [8]:
########################### Dashboard Layout ################################################################################
app.layout = html.Div([
        html.Div([html.H3('Interactive Dashboard',style={'border': '1px solid #cfcfcf',\
                                                             'text-align': 'center','background-color': '#0e508a',
                                                            'margin-top': '0px', 'margin-bottom': '0px','color': 'white'})]),
        html.Div(children=[
            html.Div(children=[
                    html.H4('3D Plot',style={'border': '1px solid #cfcfcf','background-color': '#1373c7fa',\
                                                            'margin-top': '0px', 'margin-bottom': '0px','color': 'white'}),
                    dcc.Graph(id='diff-chart',figure=fig3, style={'border': '1px solid #cfcfcf'})
            ],style={'text-align': 'center','display': 'inline-block','width': '50%'}),
            
            html.Div(children=[
                    html.H4('Main Graph',style={'border': '1px solid #cfcfcf','background-color': '#1373c7fa',
                                               'margin-top': '0px','color': 'white', 'margin-bottom': '0px'}),
                    dcc.Graph(id='example-graph',figure=fig,style={'border': '1px solid #cfcfcf'}),   
            ],style={'text-align': 'center','display': 'inline-block','width': '50%'} ),  
                
            
            
        ]),
    
    # A Slider and dropdown can be added here
        
        html.Div(children=[
            html.H4('Wind-Rose Chart',style={'width':'100%','text-align': 'center','margin-left': '0',\
                'display': 'block','background': '#e5ecf6','height': '42px', 'margin': '0px 0px 0px 0px',
                'padding-top': '18px','background': '#e5ecf6','height': '42px','padding-top': '18px'}),
            
            dcc.Graph(id='rose-chart',figure=fig2, style={'width':'100%','display': 'inline-block','margin-left': '0',
                                                     'border': '1px solid #cfcfcf',
                                                        'border-radius': '2px','height': '456.995px',
                                                        'background': '#f7f7f7',
                                                     }),],style={'text-align': 'center',\
                                                                    'display': 'inline-block','width': '45%'}),
    
        html.Div(children=[
            html.Div([html.H4('The Current Weather',style={'width':'100%','text-align': 'center','margin-left': '0',\
                'display': 'block','background': '#e5ecf6','height': '42px', 'margin': '0px 0px 0px 0px',
                'padding-top': '18px','background': '#e5ecf6','height': '42px','padding-top': '18px'})]),
            html.Div([
            html.H4(""),
            html.H6("It is Currently",style= {'margin-left': '20px','text-align': 'left'}),html.H4(Wdis,style={\
                                        'font-weight': '800','font-size': '30px'}),
            
            html.H6("Current Temperature",style= {'margin-left': '20px','text-align': 'left'}),html.H4(cTemp,style={\
                                        'font-weight': '800','font-size': '30px'}),
            html.H6("Current Wind Speed",style= {'margin-left': '20px','text-align': 'left'}),html.H4(cWindsp,style={\
                                        'font-weight': '800','font-size': '30px'}),      
            html.H6("Current Wind Direction in Degrees",style= {'margin-left': '20px','text-align': 'left'}),html.H4(cWinddir,style={\
                                        'font-weight': '800','font-size': '30px'}),],style=\
                { 'border': '1px solid #cfcfcf','padding-top': '19px','height': '436px'\
                                                                           
                                    }),
          
        
           
                            ],style={'text-align': 'center',\
                                    'border': '1px solid #cfcfcf','position': 'absolute',\
                                     'display': 'inline-block','width': '54%', } ),

                     


                            ],style={'border': '1px solid #cfcfcf'})



############################# CallsBack #######################################################################

@app.callback(
    [Output('example-graph', 'figure'),
    Output('rose-chart', 'figure'),
    Output('diff-chart', 'figure')],
    [Input('year-slider', 'value'),
    Input('measure-select', 'value')]
   )

########################## CallBack Function #################################################################

def mainGraph1(yearin,selected1):
    newYear = Dyears.get(yearin)
    run_df = df[(df.index > '11/01/{} 00:00'.format(str(years[0]))) & (df.index <= '11/01/{} 00:00'.format(newYear))]
    wind = run_df['wddir'].value_counts().to_frame()
    wind = wind.sort_index(axis=0, level=None, ascending=True,)

    thedate = run_df.index
    selected = selected1
    fig = go.Figure()
    for i in selected:
        if i == 'BoilerOn':           
            y0 = run_df['{}'.format(i)] //5
        else:
            
            y0 = run_df['{}'.format(i)]
        thename = i
        fig.add_trace(go.Scatter(x=thedate, y=y0,mode='lines',name= descript.get(i))),
    

    fig2 = go.Figure()
    fig2.add_trace(go.Barpolar(r = wind.wddir,marker_color='rgb(33,208,57)'))
    fig2.update_layout(title='Wind Speed Distribution',font_size=16,legend_font_size=16,polar_angularaxis_rotation=90,)
    
        
    return fig, fig2, fig3
    

######################################################################################################


if __name__ == '__main__':
    app.run_server( mode='inline',host="localhost",port=8051,debug=True)
    
    
# Afer running this cell please visit:  http://127.0.0.1:8051/